# Extract features from textual data using BERT and Path signature

In [1]:
# solve issue with autocomplete
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import torch
import pickle
import re

# import sys
# sys.path.insert(0, "../../timeline_generation/")  # Adds higher directory to python modules path
# import src.data_handler

/datadrive/anaconda3/envs/py38esig/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load dataset

In [3]:
# loadHF: load a dataset from hugging face
from src import loadHF
data_loader = loadHF(dataset_name="newspop", 
                     split_name="train")

In [4]:
# Note: default_preproces_newspop is implemented in loadHF
data_loader.load_preprocessed_df(default_preprocess="newspop")

[INFO] load dataframe, split: train...


/datadrive/anaconda3/envs/py38esig/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Using custom data configuration default
Found cached dataset newspop (/home/lwm/.cache/huggingface/datasets/newspop/default/0.0.0/9904d4082ffd3c0953efa538ff926c43d27da8f37c9b5d6a13f51ab96740474e)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 583.03it/s]


[INFO] preprocess...
[INFO] preprocessed dataframe can be accessed: .dataset_df


In [5]:
dataset_df = data_loader.dataset_df

In [6]:
dataset_df.head()

,content,datetime,label,timeline_id,postid
0,Obama Lays Wreath at Arlington National Cemete...,2015-01-06 00:12:00,1,11,1
1,"Tim Haywood, investment director business-unit...",2015-01-05 00:00:00,2,8,1
2,"Nouriel Roubini, NYU professor and chairman at...",2015-01-02 00:12:00,2,3,1
3,Finland's economy expanded marginally in the t...,2015-01-04 00:00:00,2,6,9
4,Tourism and public spending continued to boost...,2015-01-06 00:12:00,2,11,1


## Encode labels

In [7]:
dataset_df = dataset_df[:500]
dataset_df["label"].value_counts()

0    381
2    110
1      9
Name: label, dtype: int64

## Model specifics

Nested dictionary for models specifications.

This includes models for encoding text, path signature and etc.

In [8]:
model_specifics = {
    "encoder_args": {
        "col_name_text": "content",
        "model_name": "all-MiniLM-L6-v2",
        "model_args": {
            "batch_size": 64,
            "show_progress_bar": True,
            "output_value": 'sentence_embedding', 
            "convert_to_numpy": True,
            "convert_to_tensor": False,
            "device": None,
            "normalize_embeddings": False
        }
    },
    "dim_reduction": {
        "method": 'ppapca', #options: ppapca, ppapcappa, umap
        "num_components": 10, # options: any int number between 1 and embedding dimensions
    },
    "time_injection": {
        "history_tp": 'timestamp', #options: timestamp, None
        "post_tp": 'timestamp', #options: timestamp, timediff, None
    },
    "embedding":{
        "global_embedding_tp": 'SBERT', #options: SBERT, BERT_cls , BERT_mean, BERT_max
        "post_embedding_tp": 'sentence', #options: sentence, reduced
        "feature_combination_method": 'attention', #options concatenation, attention 
    },
    "signature": {
        "dimensions": 3, #options: any int number larger than 1
        "method": 'log', # options: log, sig
        "interval": 1/12
    },
    "classifier": {
        "classifier_name": 'FFN2hidden', # options: FFN2hidden (any future classifiers added)
        "classes_num": '3class', #options: 3class (5class to be added in the future)
    }
}

## Encode text and reduce dimensionality

In [9]:
from src import textEncoder
text_encoder = textEncoder(dataset_df,
                           col_name_text=model_specifics["encoder_args"]["col_name_text"], 
                           model_name=model_specifics["encoder_args"]["model_name"],
                           model_args=model_specifics["encoder_args"]["model_args"]
                          )

In [10]:
text_encoder.encode_sentence_transformer()

[INFO] number of sentences to encode: 500


Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.98it/s]


In [11]:
embeddings_sentence = text_encoder.embeddings_sentence

## Dimensionality reduction

In [12]:
#dimensionality reduction
from src import DimensionalityReduction

reduction = DimensionalityReduction(method= model_specifics["dim_reduction"]['method'], 
                                    components=model_specifics["dim_reduction"]['num_components'])

In [13]:
embeddings_reduced = reduction.fit_transform(embeddings_sentence)

In [14]:
print(embeddings_sentence.shape)
print(embeddings_reduced.shape)

(500, 384)
(500, 10)


In [15]:
embeddings_reduced

array([[-2.4537165e-01,  1.6304031e-01,  1.4684121e-01, ...,
        -4.2396549e-02,  2.8508076e-01,  1.6561519e-01],
       [-2.0228072e-01, -6.0418323e-02,  6.4902604e-03, ...,
         6.5756820e-02, -6.3392729e-02, -1.4671043e-01],
       [-1.1108073e-01,  9.7166961e-03,  9.3837021e-05, ...,
        -6.1412234e-02,  1.0355941e-02, -6.0100131e-02],
       ...,
       [-1.8342491e-02, -8.8214800e-03,  1.0058163e-01, ...,
         1.3878822e-01, -7.7465758e-02, -1.2908904e-01],
       [-5.0997067e-02, -2.9494038e-02,  1.4520854e-02, ...,
         9.9860460e-02, -7.3378185e-05, -9.7833090e-02],
       [-4.1051272e-02,  1.5641133e-01,  1.1480533e-01, ...,
        -2.0018610e-01, -1.5132214e-01, -3.7405938e-03]], dtype=float32)

## Time injection

In [16]:
#concatenate new dataframe
from src.dataset import get_modeling_dataframe
df = get_modeling_dataframe(dataset_df, embeddings_sentence, embeddings_reduced)

#get time features
from src.timeinjection import TimeFeatures, Padding
tf = TimeFeatures()
df = tf.get_time_features(df)


#padding
pad = Padding()
df_padded = pad.pad_timelines(df)
df_padded.shape

(12, 52, 13)

In [17]:
if (model_specifics["time_injection"]["history_tp"] == 'timestamp'):
    path = torch.from_numpy(df_padded[: , : , 2:].astype(float))
else:
    path = torch.from_numpy(df_padded[: , : , 3:].astype(float))

if (model_specifics["time_injection"]["post_tp"]== 'timestamp'):
    time_feature = torch.tensor((df[['time_encoding']].values - df['time_encoding'].mean()) / df['time_encoding'].std() )
    post_time = True
elif (model_specifics["time_injection"]["post_tp"]== 'timediff'):
    time_feature = torch.tensor( (df[['time_diff']].values - df['time_diff'].mean()) / df['time_diff'].std()  )
    post_time = True  
else: 
    time_feature = None
    post_time = False

if (model_specifics["embedding"]['post_embedding_tp'] == 'sentence'):
    bert_embeddings = torch.tensor(df[[c for c in df.columns if re.match("^e\w*[0-9]", c)]].values)
else:
    bert_embeddings = None

In [18]:
#path = path.squeeze()

## Compute signature and create features

In [19]:
#calculate paths
from src.dyadic_path import DyadicSignatures

dsig = DyadicSignatures(original_size = df.shape[0], 
                        d = path.shape[2], 
                        sig_d = model_specifics["signature"]['dimensions'],
                        intervals = model_specifics["signature"]["interval"], 
                        k_history= None, 
                        embedding_tp = model_specifics["embedding"]['post_embedding_tp'],
                        method = model_specifics["embedding"]['feature_combination_method'],
                        history_tp = model_specifics["signature"]['method'], 
                        add_time = post_time)

In [20]:
sig, last_index_dt_all = dsig.compute_signatures(path)
sig_combined = dsig.combine_signatures(sig)

In [21]:
x_data = dsig.create_features(path, sig_combined, last_index_dt_all, bert_embeddings, time_feature)

In [22]:
sig.shape, last_index_dt_all.shape, sig_combined.shape, x_data.shape

(torch.Size([12, 108, 1463]),
 torch.Size([12, 108]),
 torch.Size([12, 108, 1463]),
 torch.Size([500, 507]))

## Missing: Training classifiers, cross validation, ...